In [2]:
%load_ext cython

In [3]:
from galois.shmem import *
from galois.graphs import *

In [4]:
setActiveThreads(16)

16

## Load some graph data

In [5]:
g = LC_CSR_Graph_Directed_primitive["uint32_t", "void"]() 
# g.readGraphFromGRFile("../katana-build/inputs/reference/structured/rome99.cgr")
g.readGraphFromGRFile("/home/amp/Downloads/indochina-2004.csgr")
print(g, len(g))

<galois.graphs.LC_CSR_Graph_Directed_uint32_t_void object at 0x7ffa2efaa6f0> 7414866


## Run an extremely simple do_all parallel loop

In [6]:
%%cython
# distutils: extra_link_args=["-lgalois_shmem"]

from galois.graphs cimport *

cdef void operator(LC_CSR_Graph[uint32_t, void, dummy_true] *g, 
              LC_CSR_Graph[uint32_t, void, dummy_true].GraphNode n) nogil:
    d = &g.getData(n)
    d[0] = <uint32_t>n*<uint32_t>n

def algo(LC_CSR_Graph_Directed_uint32_t_void graph):
    g = &graph.underlying
    with nogil:
        do_all(iterate(g.begin(), g.end()), bind_leading(operator, g))

In [7]:
algo(g)
print([g.getData(i) for i in range(20)])

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]
